# 🛠️ Расширенное использование инструментов с моделями GitHub (.NET)

## 📋 Цели обучения

Этот ноутбук демонстрирует шаблоны интеграции инструментов корпоративного уровня с использованием Microsoft Agent Framework в .NET и моделями GitHub. Вы научитесь создавать сложных агентов с множеством специализированных инструментов, используя строгую типизацию C# и корпоративные возможности .NET.

**Расширенные возможности инструментов, которые вы освоите:**
- 🔧 **Архитектура с несколькими инструментами**: Создание агентов с множеством специализированных функций
- 🎯 **Типобезопасное выполнение инструментов**: Использование проверки на этапе компиляции в C#
- 📊 **Шаблоны корпоративных инструментов**: Дизайн инструментов, готовых к производству, и обработка ошибок
- 🔗 **Композиция инструментов**: Объединение инструментов для сложных бизнес-процессов

## 🎯 Преимущества архитектуры инструментов в .NET

### Особенности корпоративных инструментов
- **Проверка на этапе компиляции**: Строгая типизация гарантирует корректность параметров инструментов
- **Внедрение зависимостей**: Интеграция IoC-контейнера для управления инструментами
- **Асинхронные шаблоны**: Неблокирующее выполнение инструментов с правильным управлением ресурсами
- **Структурированное логирование**: Встроенная интеграция логирования для мониторинга выполнения инструментов

### Шаблоны, готовые к производству
- **Обработка исключений**: Полное управление ошибками с типизированными исключениями
- **Управление ресурсами**: Правильные шаблоны освобождения ресурсов и управления памятью
- **Мониторинг производительности**: Встроенные метрики и счетчики производительности
- **Управление конфигурацией**: Типобезопасная конфигурация с проверкой

## 🔧 Техническая архитектура

### Основные компоненты инструментов .NET
- **Microsoft.Extensions.AI**: Унифицированный слой абстракции инструментов
- **Microsoft.Agents.AI**: Оркестрация инструментов корпоративного уровня
- **Интеграция моделей GitHub**: Высокопроизводительный API-клиент с пулом подключений

### Конвейер выполнения инструментов
```csharp
User Request → Agent Analysis → Tool Selection → Type Validation
                 ↓               ↓              ↓
         Parameter Binding → Tool Execution → Result Processing → Response
```

## 🛠️ Категории инструментов и шаблоны

### 1. **Инструменты обработки данных**
- **Проверка ввода**: Строгая типизация с аннотациями данных
- **Операции преобразования**: Типобезопасное преобразование и форматирование данных
- **Бизнес-логика**: Инструменты для расчета и анализа, специфичные для домена
- **Форматирование вывода**: Генерация структурированных ответов

### 2. **Интеграционные инструменты**
- **API-коннекторы**: Интеграция RESTful-сервисов с HttpClient
- **Инструменты работы с базами данных**: Интеграция Entity Framework для доступа к данным
- **Операции с файлами**: Безопасные операции с файловой системой с проверкой
- **Внешние сервисы**: Шаблоны интеграции сторонних сервисов

### 3. **Утилитарные инструменты**
- **Обработка текста**: Утилиты для манипуляции строками и форматирования
- **Операции с датой/временем**: Культурно-осведомленные вычисления даты/времени
- **Математические инструменты**: Точные вычисления и статистические операции
- **Инструменты проверки**: Проверка бизнес-правил и верификация данных

## ⚙️ Предварительные требования и настройка

**Среда разработки:**
- .NET 9.0 SDK или выше
- Visual Studio 2022 или VS Code с расширением C#
- Доступ к API моделей GitHub

**Необходимые пакеты NuGet:**
```xml
<PackageReference Include="Microsoft.Extensions.AI" Version="9.9.0" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
```

**Конфигурация среды (.env файл):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

Готовы создавать агентов корпоративного уровня с мощными, типобезопасными возможностями инструментов в .NET? Давайте спроектируем профессиональные решения! 🏢⚡


In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.1"

Installed Packages Microsoft.Extensions.AI, 9.9.1

In [ ]:
#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.2

In [ ]:
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.251001.3"

Installed Packages microsoft.agents.ai, 1.0.0-preview.251001.2

In [4]:
// #r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

In [5]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [6]:
using System;
using System.ComponentModel;
using System.ClientModel;

using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using OpenAI;

In [7]:
 using DotNetEnv;

In [8]:
Env.Load("../../../.env");

In [9]:
[Description("Provides a random vacation destination.")]
static string GetRandomDestination()
{
    var destinations = new List<string>
    {
        "Paris, France",
        "Tokyo, Japan",
        "New York City, USA",
        "Sydney, Australia",
        "Rome, Italy",
        "Barcelona, Spain",
        "Cape Town, South Africa",
        "Rio de Janeiro, Brazil",
        "Bangkok, Thailand",
        "Vancouver, Canada"
    };

    var random = new Random();
    int index = random.Next(destinations.Count);
    return destinations[index];
}

In [10]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [11]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [12]:

var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [13]:
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    instructions:"You are a helpful AI Agent that can help plan vacations for customers at random destinations", tools: [AIFunctionFactory.Create((Func<string>)GetRandomDestination)]);

In [14]:
AgentThread thread = agent.GetNewThread();

In [15]:
Console.WriteLine(await agent.RunAsync("Plan me a day trip",thread));

Your day trip will be in Paris, France! Here's a proposed itinerary for your day:

### Morning
- **Breakfast at a Local Café**: Start your day with a traditional French breakfast. Try a croissant and café au lait at a local café.
- **Visit the Eiffel Tower**: Arrive early to beat the crowds. Enjoy the stunning views of the city from the top.

### Late Morning
- **Stroll Along the Seine**: Take a leisurely walk along the Seine River and enjoy the picturesque views.
- **Visit Notre-Dame Cathedral**: Explore this iconic Gothic cathedral and its stunning architecture.

### Lunch
- **Lunch at a Bistro**: Treat yourself to a classic French lunch at a nearby bistro. Consider trying coq au vin or a delicious quiche.

### Afternoon
- **Explore the Louvre**: Spend your afternoon at the Louvre Museum. While you may not see everything, be sure to check out the Mona Lisa and other famous artworks.
- **Walk Through the Tuileries Garden**: After the museum, relax in the beautiful Tuileries Garden.

#

In [16]:
Console.WriteLine(await agent.RunAsync("I don't like that destination. Plan me another vacation.",thread));

Your new vacation destination is Rio de Janeiro, Brazil! Here's a proposed itinerary for your trip:

### Day 1: Arrival in Rio
- **Check-in**: Arrive and settle into your accommodation.
- **Evening at Copacabana Beach**: Stroll along the famous Copacabana Beach. Enjoy the vibrant atmosphere and watch the sunset.

### Day 2: Exploring the City
- **Visit Christ the Redeemer**: Start your day with a visit to the iconic Christ the Redeemer statue. Take the train up to the top for breathtaking views.
- **Explore Santa Teresa**: Wander through the charming streets of Santa Teresa, known for its colonial-style houses, art studios, and local shops.
- **Lunch at a Local Restaurant**: Enjoy traditional Brazilian cuisine, such as feijoada (black bean stew).
- **Afternoon at Sugarloaf Mountain**: Take a cable car ride to Sugarloaf Mountain for panoramic views of the city and coastline.

### Day 3: Cultural Experience
- **Visit the Selarón Steps**: Walk up the colorful Selarón Steps, an iconic mosa


---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
